In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import time
import itertools
import pandas as pd

random.seed(42)

In [5]:
def calcular_distancia_euclidiana(origem, destino):
    return math.sqrt((destino[0] - origem[0])**2 + (destino[1] - origem[1])**2)


In [6]:
def forca_bruta_steiner(G, origem, terminais):
    menor_custo = math.inf
    melhor_arvore = None

    for tamanho_subgrafo in range(2, len(terminais) + 1):  # Subconjuntos de tamanho 2 até o tamanho total
        for subgrafo in itertools.combinations(terminais, tamanho_subgrafo):
            # Construir uma árvore para o subconjunto atual
            arvore_atual = nx.Graph()

            for origem, destino in itertools.combinations(subgrafo, 2):
                caminho = nx.shortest_path(G, source=origem, target=destino, weight='peso')
                for i in range(len(caminho) - 1):
                    arvore_atual.add_edge(caminho[i], caminho[i+1], peso=G.edges[caminho[i], caminho[i+1]]['peso'])

            # Calcular o custo total da árvore
            custo_atual = sum(G.edges[edge]['peso'] for edge in arvore_atual.edges)

            # Atualizar se o custo atual for menor
            if custo_atual < menor_custo:
                menor_custo = custo_atual
                melhor_arvore = arvore_atual.copy()

    return melhor_arvore

In [9]:
def arvore_steiner(quantidade_pontos, quantidade_terminais):
    pontos = []
    for _ in range(quantidade_pontos):
        x = random.randint(0, 100)  # Gera um número aleatório entre 0 e 100 para a coordenada x
        y = random.randint(0, 100)  # Gera um número aleatório entre 0 e 100 para a coordenada y
        pontos.append((x, y))

    G = nx.Graph() # Grafo original
    G.add_nodes_from(pontos) # Adiciona o conjunto de pontos

    terminais = random.sample(list(G.nodes()), quantidade_terminais)


    G = nx.Graph() # Grafo original
    G.add_nodes_from(pontos) # Adiciona o conjunto de pontos
    # Fazer a busca entre a origem e todos os terminais, um a um
    # Transforma os pontos originais em um grafo completo
    for i, origem in enumerate(G.nodes()):
        for j, destino in enumerate(G.nodes()):
            if(j > i):
                distancia = calcular_distancia_euclidiana(origem, destino)
                G.add_edge(origem, destino, peso=distancia)
                
    origem = terminais[0]

    tempo_inicio = time.time()
    min_arvore_steiner = forca_bruta_steiner(G, origem, terminais)
    tempo_total = time.time() - tempo_inicio

    return {'pontos': quantidade_pontos, 'terminais': quantidade_terminais, 'tempo': tempo_total}

In [36]:
analise_tempo = []
qtd_pontos = 1000
sair = False
tempo_limite = 5.0

for i in range(1, qtd_pontos, 5):
    if(sair):
        break
    for j in range(1, i, 3):
        print(i, j)
        analise_tempo.append(arvore_steiner(i, j))
        print(analise_tempo[-1])
        if(analise_tempo[-1]['tempo'] > tempo_limite):
            sair = True
            break

analise = pd.DataFrame(analise_tempo)
analise.columns = ['Pontos', 'Terminais', 'tempo_1']

qtd_pontos = i

# rodar a mesma analise mais duas vezes para tirar a media dos tempos
for k in range(2, 4):
    analise_tempo = []
    for i in range(1, qtd_pontos, 5):
        for j in range(1, i, 3):
            print(i, j)
            analise_tempo.append(arvore_steiner(i, j))

    analise_parcial = pd.DataFrame(analise_tempo)
    analise_parcial.columns = ['Pontos', 'Terminais', 'Tempo']
    analise[f'tempo_{k}'] = analise_parcial['Tempo']

analise['Media_Tempo'] = (analise['tempo_1'] + analise['tempo_2'] + analise['tempo_3'])/3
analise.to_csv('Analise_tempo_foca_bruta.csv')


6 1
{'pontos': 6, 'terminais': 1, 'tempo': 0.0}
6 4
{'pontos': 6, 'terminais': 4, 'tempo': 0.0010089874267578125}
11 1
{'pontos': 11, 'terminais': 1, 'tempo': 0.0}
11 4
{'pontos': 11, 'terminais': 4, 'tempo': 0.002000570297241211}
11 7
{'pontos': 11, 'terminais': 7, 'tempo': 0.05566573143005371}
11 10
{'pontos': 11, 'terminais': 10, 'tempo': 0.8849914073944092}
16 1
{'pontos': 16, 'terminais': 1, 'tempo': 0.0}
16 4
{'pontos': 16, 'terminais': 4, 'tempo': 0.002999544143676758}
16 7
{'pontos': 16, 'terminais': 7, 'tempo': 0.09278750419616699}
16 10
{'pontos': 16, 'terminais': 10, 'tempo': 1.4027049541473389}
16 13
{'pontos': 16, 'terminais': 13, 'tempo': 18.934545278549194}
